# 🔵 Lesson 7: Schedulers Explained

If the U-Net is the engine, the **Scheduler** is the transmission.

The process of denoising is mathematically an **Ordinary Differential Equation (ODE)**.
$$ dx = -\dot{\sigma}(t) \sigma(t) \nabla_x \log p_t(x) dt $$

Schedulers are just algorithms to solve this equation.

### Types:
1.  **Euler Ancestral (Euler A)**: The standard. Adds random noise at each step. "Creative" but unstable.
2.  **DDIM**: Deterministic. Good for inverting images.
3.  **DPM++ 2M Karras**: Uses 2nd order derivatives. Very fast (20 steps). High quality.

In [ ]:
# 1. Setup
import notebook_utils
project_root, device, dtype = notebook_utils.setup_notebook()

from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler, DPMSolverMultistepScheduler, DDIMScheduler
import torch
import matplotlib.pyplot as plt

## 1. Load Pipeline
We will load one pipeline and swap out the schedulers.

In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=dtype).to(device)
print("Pipeline Locked.")

## 2. The Shootout

We will generate the **exact same seed** with 3 different math solvers.

In [ ]:
prompt = "a portrait of a wizard, detailed oil painting"
seed = 12345
steps = 20

results = []
names = ["Euler Ancestral", "DDIM", "DPM++ 2M Karras"]

# 1. Euler A
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
gen = torch.Generator(device).manual_seed(seed)
img1 = pipe(prompt, num_inference_steps=steps, generator=gen).images[0]
results.append(img1)

# 2. DDIM
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
gen = torch.Generator(device).manual_seed(seed)
img2 = pipe(prompt, num_inference_steps=steps, generator=gen).images[0]
results.append(img2)

# 3. DPM++
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config, use_karras_sigmas=True)
gen = torch.Generator(device).manual_seed(seed)
img3 = pipe(prompt, num_inference_steps=steps, generator=gen).images[0]
results.append(img3)

# Visualize
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
for i, (img, name) in enumerate(zip(results, names)):
    axs[i].imshow(img)
    axs[i].set_title(name)
    axs[i].axis('off')
plt.show()

## 3. Analysis

- **Euler A**: Often looks softer or more "dreamy". It changes slightly even with the same seed because it adds noise (ancestral sampling).
- **DDIM**: Very sharp, but needs more steps usually (20 steps might be grainy).
- **DPM++**: By far the best quality at low steps. It uses advanced calculus to predict the curve of the noise, getting to the answer faster.